# Advent of Code 2025

> Computer science is no more about computers than astronomy is about telescopes.
>
-- Edsger W. Dijkstra

## Imports and definitions

In [1]:
from urllib import request


def aocin(day):
    try:
        with open(f'input/{day}') as f:
            return f.read().strip()
    except FileNotFoundError:
        r = request.Request(f'https://adventofcode.com/2025/day/{day}/input')
        r.add_header('Cookie', open('../.aoccookie').read().strip())
        r.add_header('User-Agent', 'github.com/edoannunziata/jardin')
        with open(f'input/{day}', 'bw') as f:
            f.write(request.urlopen(r).read())
        with open(f'input/{day}') as f:
            return f.read().strip()

## [Day 1: ??](https://adventofcode.com/2025/day/1)

In [2]:
A = 0
assert A == 0